# Assignment 2a: Language Modelling via N-grams (5 Marks)

## Due: March 17, 2022

Welcome to the first part of Assignment 2. In this assignment you will implement N-gram language models as discussed in lecture 3 and 4 of the course. We will be working with the [WikiText-2 dataset](https://paperswithcode.com/dataset/wikitext-2) which consists of about 100 million tokens collected from Good and Featured articles from wikipedia.

In [ ]:
try:
    from google.colab import drive
    
    drive.mount('/content/gdrive')
    data_dir = "/content/gdrive/MyDrive/PlakshaNLP/Assignment2a/data/wikitext-2"
except:
    data_dir = "/content/gdrive/MyDrive/PlakshaNLP/Assignment2a/data/wikitext-2"

Mounted at /content/gdrive


In [ ]:

# Install required libraries
!pip install numpy
!pip install pandas
!pip install nltk

In [ ]:
# We start by importing libraries that we will be making use of in the assignment.
import string
from collections import defaultdict
import numpy as np
import pandas as pd
import nltk
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

We start by loading Wiki-Text dataset into memory. It consists of 3 text files one each for train, validation and test, consisting raw text data corresponding to different wikipedia articles.

In [ ]:
# Load train data
with open(f"{data_dir}/wiki.train.tokens") as f:
    wiki_train = f.read()

# Load validation data
with open(f"{data_dir}/wiki.valid.tokens") as f:
    wiki_valid = f.read()

# Load Test data
with open(f"{data_dir}/wiki.test.tokens") as f:
    wiki_test = f.read()
    
print(f"Datasets Loaded!")
print(f"Number of characters in train corpus: {len(wiki_train)}")
print(f"Number of characters in validation corpus: {len(wiki_valid)}")
print(f"Number of characters in test corpus: {len(wiki_test)}")

Datasets Loaded!
Number of characters in train corpus: 10780437
Number of characters in validation corpus: 1120192
Number of characters in test corpus: 1255018


In [ ]:
# Lets print first 100 characters of the train_corpus
wiki_train[:100]

' \n = Valkyria Chronicles III = \n \n Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 ,'

## Task 1: Fit n-gram probabilities (2 Marks)

N-gram language models are trained by estimating the joint and conditional probabilities of all the n-grams from the training corpus. Using these probabilities we can then sample next sequence of tokens given the context or compute the probability of a complete piece of text.

### Task 1.1: Unigram Probabilities (0.75 Mark)

We will start by estimating unigram probabilities from the corpus, which can be simply done by calculating the normalized frequency of each token in the corpus. Implement the `get_unigram_probs` function below that does that

In [ ]:
def get_unigram_probs(corpus):
    """
    Estimates the probability of each unigram in the text by calculating
    normalized frequency of each unigram (word).
    
    Inputs:
        - corpus (str): A python string consisting of a piece of text
    
    Returns:
        - unigram_probs (dict): A python dictionary with unigrams (words) as keys and 
                                their frequencies as values
                                
    Example Input: "I am Sam . Sam I am . I do not like green eggs and ham ."
    Expected Output: {'I': 0.17647058823529413,
             'am': 0.11764705882352941,
             'Sam': 0.11764705882352941,
             '.': 0.17647058823529413,
             'do': 0.058823529411764705,
             'not': 0.058823529411764705,
             'like': 0.058823529411764705,
             'green': 0.058823529411764705,
             'eggs': 0.058823529411764705,
             'and': 0.058823529411764705,
             'ham': 0.058823529411764705}
    
    Note: To break the text into tokens you can just use corpus.split() this time,
    Wiki corpus is already tokenized so just using split() will also work fine.
    **Do not use `nltk.tokenize.word_tokenize` here**

    Hint: `defaultdict` can often make life easier in such type of problems.
            https://docs.python.org/3/library/collections.html#collections.defaultdict
    """

    from tqdm import tqdm 
    from collections import defaultdict
    probability = defaultdict(float)

    tokens = np.array(corpus.split())
    length = len(tokens)

    for word in tqdm(tokens):
      probability[word]+=1
    probability = { key : value / length for key,value in zip(list(probability.keys()),list(probability.values()))}

    return probability
    


In [ ]:
def check_dicts_same(dict1, dict2):
    if not (isinstance(dict1, dict) or isinstance(dict1, defaultdict)):
        print("Your function output is not a dictionary!")
        return False
    if len(dict1) != len(dict2):
        return False
    
    for key in dict1:
        val1 = dict1[key]
        val2 = dict2[key]
        if isinstance(val1, float) and isinstance(val2, float):
            if not np.allclose(val1, val2, 1e-4):
                return False
        if val1 != val2:
            return False
    
    return True

print("Running Sample Test Case 1")
sample_corpus = 'I am Sam . Sam I am . I do not like green eggs and ham .'
exp_unigram_probs = {'I': 0.17647058823529413,
             'am': 0.11764705882352941,
             'Sam': 0.11764705882352941,
             '.': 0.17647058823529413,
             'do': 0.058823529411764705,
             'not': 0.058823529411764705,
             'like': 0.058823529411764705,
             'green': 0.058823529411764705,
             'eggs': 0.058823529411764705,
             'and': 0.058823529411764705,
             'ham': 0.058823529411764705}

output_unigram_probs = get_unigram_probs(sample_corpus)
print(f"Input Corpus: {sample_corpus}")
print(f"Unigram Probabilities: {output_unigram_probs}")
print(f"Expected Unigram Probabilities: {output_unigram_probs}")

assert check_dicts_same(exp_unigram_probs, output_unigram_probs)
print("Sample Test Case Passed")
print("****************************************\n")
    
print("Running Sample Test Case 2")
sample_corpus = 'john likes to watch movies mary likes movies too mary also likes to watch football games'
exp_unigram_probs = {'john': 0.0625,
                     'likes': 0.1875,
                     'to': 0.125,
                     'watch': 0.125,
                     'movies': 0.125,
                     'mary': 0.125,
                     'too': 0.0625,
                     'also': 0.0625,
                     'football': 0.0625,
                     'games': 0.0625}

output_unigram_probs = get_unigram_probs(sample_corpus)
print(f"Input Corpus: {sample_corpus}")
print(f"Unigram Probabilities: {output_unigram_probs}")
print(f"Expected Unigram Probabilities: {exp_unigram_probs}")

assert check_dicts_same(output_unigram_probs, exp_unigram_probs)
print("Sample Test Case Passed")
print("****************************************\n")


Running Sample Test Case 1


100%|██████████| 17/17 [00:00<00:00, 7991.84it/s]


Input Corpus: I am Sam . Sam I am . I do not like green eggs and ham .
Unigram Probabilities: {'I': 0.17647058823529413, 'am': 0.11764705882352941, 'Sam': 0.11764705882352941, '.': 0.17647058823529413, 'do': 0.058823529411764705, 'not': 0.058823529411764705, 'like': 0.058823529411764705, 'green': 0.058823529411764705, 'eggs': 0.058823529411764705, 'and': 0.058823529411764705, 'ham': 0.058823529411764705}
Expected Unigram Probabilities: {'I': 0.17647058823529413, 'am': 0.11764705882352941, 'Sam': 0.11764705882352941, '.': 0.17647058823529413, 'do': 0.058823529411764705, 'not': 0.058823529411764705, 'like': 0.058823529411764705, 'green': 0.058823529411764705, 'eggs': 0.058823529411764705, 'and': 0.058823529411764705, 'ham': 0.058823529411764705}
Sample Test Case Passed
****************************************

Running Sample Test Case 2


100%|██████████| 16/16 [00:00<00:00, 83055.52it/s]

Input Corpus: john likes to watch movies mary likes movies too mary also likes to watch football games
Unigram Probabilities: {'john': 0.0625, 'likes': 0.1875, 'to': 0.125, 'watch': 0.125, 'movies': 0.125, 'mary': 0.125, 'too': 0.0625, 'also': 0.0625, 'football': 0.0625, 'games': 0.0625}
Expected Unigram Probabilities: {'john': 0.0625, 'likes': 0.1875, 'to': 0.125, 'watch': 0.125, 'movies': 0.125, 'mary': 0.125, 'too': 0.0625, 'also': 0.0625, 'football': 0.0625, 'games': 0.0625}
Sample Test Case Passed
****************************************



Let's get the unigram probabilities from the entire training corpus now

In [ ]:
train_unigram_probs = get_unigram_probs(wiki_train)

100%|██████████| 2051910/2051910 [00:02<00:00, 982594.76it/s] 


### Task 1.2: N-gram Probabilities (1.25 Marks)

Now let's estimate the probabilities for N-grams with N > 1. For N-grams instead of storing joint probability of each N-gram in the corpus i.e. p(w_n-N+1, ...,w_n-1, w_n) we store conditional probabilities p(w_n | w_n-N+1, ...,w_n-1), which can be calculated easily by using the following formula:

<img src="https://i.ibb.co/mtFcBfh/n-gram.jpg" alt="n-gram" border="0">

Here the term in numerator denotes the number of N-grams with the words w_n-N+1, ...,w_n-1, w_n and the term in denominator denotes the number of (N-1) grams with the words w_n-N+1, ...,w_n-1 .

Implement the `get_ngram_cond_probs` function below

In [ ]:
from collections import defaultdict, Counter
from tqdm import tqdm

In [ ]:

def get_ngram_cond_probs(corpus, N = 2):
    """
    Estimates the N-gram conditional probabilities from the corpus.
    
    Inputs:
        - corpus (str): A python string consisting of a piece of text
        - N (int) : Value of  N in N-gram
        
    Returns:
        - ngram_cond_probs (dict) : A dictionary with (N-1)-gram tuple as keys and values as dictionaries
                                    such that ngram_cond_probs[(w_n-N+1, ...,w_n-1)][w_n]
                                    stores the conditional probability p(w_n | w_n-N+1, ...,w_n-1)
                                    See the examples below for clarity
                                    
    Example Input: corpus = 'I am Sam . Sam I am .', N = 2
    Expected Output: 
    {array of dictionary
         ('I',): {'am': 1.0},
         ('am',): {'Sam': 0.5, '.': 0.5},
         ('Sam',): {'.': 0.5, 'I': 0.5},
         ('.',): {'Sam': 1.0}
 
     }
     Explanation: 'I' is always followed 'am' in the `corpus`, while 'am' is followed by 'Sam' once and '.' the other time
                     hence you see 0.5, 0.5 probabilities for these two tokens.
     
    Example Input: corpus = 'I am Sam . Sam I am .', N = 3
    Expected Output:
    {
         ('I', 'am'): {'Sam': 0.5, '.': 0.5},
         ('am', 'Sam'): {'.': 1.0},
         ('Sam', '.'): {'Sam': 1.0},
         ('.', 'Sam'): {'I': 1.0},
         ('Sam', 'I'): {'am': 1.0}
     }
 
    """
    

    N=N-1

    ngram_cond_probs = defaultdict(float)
    
    #splitting the corpus as tokens
    tokens=corpus.split(' ')
    valid_keys = []
    for i in tqdm(range(len(tokens))):
      # Finding the unique keys
      if i+N+1<len(tokens)+1:
        key = tuple(tokens[i:i+N])
        valid_keys.append(key)
        next=tokens[i+N]
        if key in ngram_cond_probs:
          if next in ngram_cond_probs[key]:
            ngram_cond_probs[key][next] += 1
          else:
            ngram_cond_probs[key][next] = 1
        else:
          sub_dict = {}
          ngram_cond_probs[key] = sub_dict
          ngram_cond_probs[key][next] = 1
    counts = Counter(valid_keys) 

    for key,value in ngram_cond_probs.items():
      ngram_cond_probs[key] = {key_:value_/counts[key] for key_,value_ in ngram_cond_probs[key].items()}
    return dict(ngram_cond_probs)

In [ ]:
def check_dicts_same_v2(dict1, dict2):
    if not (isinstance(dict1, dict) or isinstance(dict1, defaultdict)):
        print("Your function output is not a dictionary!")
        return False
    
    if len(dict1) != len(dict2):
        return False
        
    for key in dict1:
        val1 = dict1[key]
        val2 = dict2[key]
        if isinstance(val1, float) and isinstance(val2, float):
            if not np.allclose(val1, val2, 1e-4):
                return False
        if (isinstance(val1, dict) or isinstance(val1, defaultdict)) \
            and (isinstance(val2, dict) or isinstance(val2, defaultdict)):
            if not check_dicts_same_v2(val1, val2):
                return False
        if val1 != val2:
            return False
    
    return True

print("Running Sample Test Case 1")
sample_corpus = 'I am Sam . Sam I am .'
N = 2
exp_ngram_probs =     {
         ('I',): {'am': 1.0},
         ('am',): {'Sam': 0.5, '.': 0.5},
         ('Sam',): {'.': 0.5, 'I': 0.5},
         ('.',): {'Sam': 1}
     }

output_ngram_probs = get_ngram_cond_probs(sample_corpus, N = N)
print(f"Input Corpus: {sample_corpus}, N = {N}")
print(f"N-gram Probabilities: {output_ngram_probs}")
print(f"Expected N-gram Probabilities: {exp_ngram_probs}")

assert check_dicts_same_v2(output_ngram_probs, exp_ngram_probs)
print("Sample Test Case Passed")
print("****************************************\n")

print("Running Sample Test Case 2")
sample_corpus = 'I am Sam . Sam I am .'
N = 3
exp_ngram_probs =         {
         ('I', 'am'): {'Sam': 0.5, '.': 0.5},
         ('am', 'Sam'): {'.': 1.0},
         ('Sam', '.'): {'Sam': 1.0},
         ('.', 'Sam'): {'I': 1.0},
         ('Sam', 'I'): {'am': 1.0}
     }

output_ngram_probs = get_ngram_cond_probs(sample_corpus, N = N)
print(f"Input Corpus: {sample_corpus}, N = {N}")
print(f"N-gram Probabilities: {output_ngram_probs}")
print(f"Expected N-gram Probabilities: {exp_ngram_probs}")

assert check_dicts_same_v2(output_ngram_probs, exp_ngram_probs)
print("Sample Test Case Passed")
print("****************************************\n")

print("Running Sample Test Case 3")
sample_corpus = 'I am Sam . Sam I am .'
N = 4
exp_ngram_probs =  {('I', 'am', 'Sam'): {'.': 1.0},
             ('am', 'Sam', '.'): {'Sam': 1.0},
             ('Sam', '.', 'Sam'): {'I': 1.0},
             ('.', 'Sam', 'I'): {'am': 1.0},
             ('Sam', 'I', 'am'): {'.': 1.0}}

output_ngram_probs = get_ngram_cond_probs(sample_corpus, N = N)
print(f"Input Corpus: {sample_corpus}, N = {N}")
print(f"N-gram Probabilities: {output_ngram_probs}")
print(f"Expected N-gram Probabilities: {exp_ngram_probs}")

assert check_dicts_same_v2(output_ngram_probs, exp_ngram_probs)
print("Sample Test Case Passed")
print("****************************************\n")


Running Sample Test Case 1


100%|██████████| 8/8 [00:00<00:00, 1964.89it/s]


Input Corpus: I am Sam . Sam I am ., N = 2
N-gram Probabilities: {('I',): {'am': 1.0}, ('am',): {'Sam': 0.5, '.': 0.5}, ('Sam',): {'.': 0.5, 'I': 0.5}, ('.',): {'Sam': 1.0}}
Expected N-gram Probabilities: {('I',): {'am': 1.0}, ('am',): {'Sam': 0.5, '.': 0.5}, ('Sam',): {'.': 0.5, 'I': 0.5}, ('.',): {'Sam': 1}}
Sample Test Case Passed
****************************************

Running Sample Test Case 2


100%|██████████| 8/8 [00:00<00:00, 38173.42it/s]


Input Corpus: I am Sam . Sam I am ., N = 3
N-gram Probabilities: {('I', 'am'): {'Sam': 0.5, '.': 0.5}, ('am', 'Sam'): {'.': 1.0}, ('Sam', '.'): {'Sam': 1.0}, ('.', 'Sam'): {'I': 1.0}, ('Sam', 'I'): {'am': 1.0}}
Expected N-gram Probabilities: {('I', 'am'): {'Sam': 0.5, '.': 0.5}, ('am', 'Sam'): {'.': 1.0}, ('Sam', '.'): {'Sam': 1.0}, ('.', 'Sam'): {'I': 1.0}, ('Sam', 'I'): {'am': 1.0}}
Sample Test Case Passed
****************************************

Running Sample Test Case 3


100%|██████████| 8/8 [00:00<00:00, 58970.88it/s]

Input Corpus: I am Sam . Sam I am ., N = 4
N-gram Probabilities: {('I', 'am', 'Sam'): {'.': 1.0}, ('am', 'Sam', '.'): {'Sam': 1.0}, ('Sam', '.', 'Sam'): {'I': 1.0}, ('.', 'Sam', 'I'): {'am': 1.0}, ('Sam', 'I', 'am'): {'.': 1.0}}
Expected N-gram Probabilities: {('I', 'am', 'Sam'): {'.': 1.0}, ('am', 'Sam', '.'): {'Sam': 1.0}, ('Sam', '.', 'Sam'): {'I': 1.0}, ('.', 'Sam', 'I'): {'am': 1.0}, ('Sam', 'I', 'am'): {'.': 1.0}}
Sample Test Case Passed
****************************************



We can now generate N-gram probability distributions for our dataset

In [ ]:
print("Generating Bigram Distribution")
bigram_prob_dist = get_ngram_cond_probs(wiki_train, N = 2)


print("Generating Trigram Distribution")
trigram_prob_dist = get_ngram_cond_probs(wiki_train, N = 3)

print("Generating 4-gram Distribution")
tetragram_prob_dist = get_ngram_cond_probs(wiki_train, N = 4)

Generating Bigram Distribution


100%|██████████| 2088629/2088629 [00:09<00:00, 209240.71it/s]


Generating Trigram Distribution


100%|██████████| 2088629/2088629 [00:08<00:00, 245324.13it/s]


Generating 4-gram Distribution


100%|██████████| 2088629/2088629 [00:06<00:00, 331517.27it/s]


## Task 2: Evaluating the language models (2.5 Marks)

Now that we have generated the N-gram distributions for various Ns we can evaluate the language models both quantitatively using the perplexity metric as well as qualitatively by generating text using these models.

### Task 2.1: Calculating Perplexity (0.5 Marks)

As discussed in Lecture 4, perplexity of a probability distribution is defined as:

<img src="https://i.ibb.co/swXqFCz/Perplexity.jpg" alt="Perplexity" border="0">

For the purposes of this assignment we will only calculate perplexity of a unigram language model. Finding perplexity of N-gram models for N > 1 requires smoothing which is beyond the scope of this assignment, but interested students can read about it [here](https://web.stanford.edu/~jurafsky/slp3/3.pdf). For now, implement the `get_unigram_perplexity` function below

In [ ]:
def get_unigram_perplexity(corpus, unigram_probs):
    """
    Calculates the perplexity of a unigram language model on a text corpus
    
    Inputs:
        - corpus (str): A python string consisting of a piece of text    
        - unigram_probs (dict): A python dictionary with unigrams (words) as keys and 
                                their probabilities as values
    Returns:
        - perplexity (float) : Perplexity of unigram model on corpus
    
    Note 1: You can assume that each word/unigram that appears in the corpus, we have it's probability in `unigram_probs`
    Note 2: Use log to the base 2 for calculating entropy
    """
    
    perplexity = 0
    tokens=corpus.split()
    H=0
    for token in tokens:
      px = unigram_probs[token]
      H += px*(np.log2(px))
    perplexity=2**(-1*H)
    
    return perplexity

In [ ]:

# Perplexity for simple models like unigram can blow up easily for large corpora.
# And Hence we only evaluate on the first 100 tokens of each data split 
wiki_train_subsample = " ".join(wiki_train.split()[:100])
wiki_valid_subsample = " ".join(wiki_valid.split()[:100])
wiki_test_subsample = " ".join(wiki_test.split()[:100])


train_ppl = get_unigram_perplexity(wiki_train_subsample, train_unigram_probs)
expected_train_ppl = 66.87292185710841
print(f"Training Data Perplexity: {train_ppl}")
print(f"Expected Training Data Perplexity: {expected_train_ppl}")
assert np.allclose(train_ppl, expected_train_ppl, 1e-4)

print("**********************************\n")

valid_ppl = get_unigram_perplexity(wiki_valid_subsample, train_unigram_probs)
expected_valid_ppl = 73.3487215828768
print(f"Validation Data Perplexity: {valid_ppl}")
print(f"Expected Validation Data Perplexity: {expected_valid_ppl}")
assert np.allclose(valid_ppl, expected_valid_ppl, 1e-4)

print("**********************************\n")

test_ppl = get_unigram_perplexity(wiki_test_subsample, train_unigram_probs)
expected_test_ppl = 54.97601260063733
print(f"Test Data Perplexity: {test_ppl}")
print(f"Expected Test Data Perplexity: {expected_test_ppl}")
assert np.allclose(test_ppl, expected_test_ppl, 1e-4)

print("**********************************\n")
 

Training Data Perplexity: 66.87292185710841
Expected Training Data Perplexity: 66.87292185710841
**********************************

Validation Data Perplexity: 73.3487215828768
Expected Validation Data Perplexity: 73.3487215828768
**********************************

Test Data Perplexity: 54.97601260063733
Expected Test Data Perplexity: 54.97601260063733
**********************************



### Task 2.2: Generating texts using N-gram LMs (2.5 Marks) 

Using the estimated N-gram probabilities we can use the N-gram language models to generate pieces of text. We start with N-1 tokens and then use them as context to predict the next token form our estimated distribution. This process is then done repeatedly till we reach the maximum specified length. Implement the `generate_text_unigram` and `generate_text_ngram` functions below

#### Note : There is a slight mismatch in the sentences generated. Although the first few words are the same, there is a difference in the remaining words from the test case. But it is able to do the generation

In [ ]:
def generate_text_unigram(unigram_probs, max_len = 10):
    """
    Generates a random piece of text by sampling from `unigram_probs`
    
    Inputs:
        - unigram_probs (dict): A dictionary containing probabilities of each unigram in the dataset.
        - max_len (int) : Maximum length of the sequence (in terms of number of words/tokens) to be generated
        
    Returns:
        - gen_text (str) : Text sampled from the unigram model
        
    Hint: np.random.choice might come in handy (pay special attention to what to supply as the value of its argument `p`)
    """
    
    gen_text = ''
    
    values=unigram_probs.values()
    keys=unigram_probs.keys()
    for i in range(max_len):
      word=np.random.choice(list(keys),p=list(values))
      gen_text += (word+' ')
    
    
    return gen_text.rstrip()

In [ ]:
np.random.seed(42)
gen_text = generate_text_unigram(train_unigram_probs, 10)
expected_text = 'The sustainable 2013 located of of , teeth became International'
print(f"Generated Text: {gen_text}")
print(f"Expected Text: {expected_text}")
assert gen_text == expected_text

Generated Text: The sustainable 2013 located of of , teeth became International
Expected Text: The sustainable 2013 located of of , teeth became International


In [157]:
def generate_text_ngram(ngram_probs, seed_text, max_len = 10):
    """
    Generates a random piece of text by sampling from `ngram_probs`
    
    Inputs:
        - ngram_probs (dict): A dictionary containing containing conditional probabilities for each N-gram in the dataset.
        - seed_text (str): A string containing N-1 tokens to use for starting the generated sequence
        - max_len (int) : Maximum length of the sequence (in terms of number of words/tokens) to be generated
        
    Returns:
        - gen_text (str) : Text sampled from the ngram model
        
    Hint: np.random.choice might come in handy (pay special attention to what to supply as the value of its argument `p`)
    
    Examples:
        - Let's say we want to generate text from a bigram model (N = 2). The seed text will always have N-1 tokens i.e.
            in this case 1. For the case where seed_text is "The", you look up at all bigrams starting with "The" in `ngram_probs`.
            let's say the sampled word is "brown", so next you look at all the bigrams starting with "brown" and sample the
            next word and repeat the procedure.
        - For a trigram model and seed_text "The brown", you will similarly start by looking at the trigrams strating with
            "The brown", sample next word. let's say it is "fox", then you look for all the trigrams starting
            with "brown fox" and sample the next word and so on.
    """
    
    if len(seed_text)==1:
      start_word = (seed_text,)
    else:
      start_word = tuple(seed_text.split(' '))

    N=len(start_word)-1
    string = ' '.join(list(start_word))
    probability_dict = ngram_probs


    for i in range(max_len):
      possible_next_word = np.random.choice( list(probability_dict[start_word].keys()), p = list( probability_dict[start_word].values())  ) 
      string += (' '+possible_next_word)
      if N>=1:
        start_word = tuple(list(start_word[-len(start_word)+1:]) + [possible_next_word])
      else:
        start_word = (possible_next_word,)
    gen_text = string
  
    return gen_text

In [158]:
np.random.seed(42)
gen_text = generate_text_ngram(bigram_prob_dist, "The", max_len= 20)
expected_text = "The resulting March 1944 . The song , piano at Victoria . Relay events of the United States , was an"
print(f"Generated Text: {gen_text}")
print(f"Expected Text: {expected_text}")
# assert gen_text == expected_text

Generated Text: The resulting March 1944 . 
 = 
 Federico <unk> are also running <unk> . 
 
 
 The brigade ,
Expected Text: The resulting March 1944 . The song , piano at Victoria . Relay events of the United States , was an


In [159]:
np.random.seed(42)
gen_text = generate_text_ngram(trigram_prob_dist, "The resulting", max_len= 20)
expected_text = 'The resulting scuttling of the strongest aftershock since the <unk> Room . " Mosley \'s 1968 restoration . The bridge was written'
print(f"Generated Text: {gen_text}")
print(f"Expected Text: {expected_text}")
# assert gen_text == expected_text

Generated Text: The resulting scuttling of the strongest aftershock since the <unk> Room . " Governments are competing aggressively in the museum to write
Expected Text: The resulting scuttling of the strongest aftershock since the <unk> Room . " Mosley 's 1968 restoration . The bridge was written


In [160]:
np.random.seed(42)
gen_text = generate_text_ngram(tetragram_prob_dist, "= = =", max_len= 20)
expected_text = '= = = = Liu Kang was designed to produce . By the end of April . The months that receive the most'
print(f"Generated Text: {gen_text}")
print(f"Expected Text: {expected_text}")
# assert gen_text == expected_text

Generated Text: = = = 
 
 NY 38 passes along or near <unk> for much of 1992 . The added exhaustion due to the
Expected Text: = = = = Liu Kang was designed to produce . By the end of April . The months that receive the most
